## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "dbfs:/FileStore/shared_uploads/rivverg@gmail.com/Bookings.csv"
file_location_facilities = "dbfs:/FileStore/shared_uploads/rivverg@gmail.com/Facilities.csv"
file_location_members = "dbfs:/FileStore/shared_uploads/rivverg@gmail.com/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: string (nullable = true)



### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
dbutils.fs.rm("dbfs:/user/hive/warehouse/country_club.db/bookings", True)
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
dbutils.fs.rm("dbfs:/user/hive/warehouse/country_club.db/facilities", True)
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
dbutils.fs.rm("dbfs:/user/hive/warehouse/country_club.db/members", True)
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03 11:00:00,2
1,4,1,2012-07-03 08:00:00,2
2,6,0,2012-07-03 18:00:00,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
facilities_df.describe().show()

+-------+------------------+---------------+------------------+------------------+------------------+------------------+
|summary|             facid|           name|        membercost|         guestcost|     initialoutlay|monthlymaintenance|
+-------+------------------+---------------+------------------+------------------+------------------+------------------+
|  count|                 9|              9|                 9|                 9|                 9|                 9|
|   mean|               4.0|           null|3.6999999999999997|28.666666666666668|4018.8888888888887|             730.0|
| stddev|2.7386127875258306|           null| 4.108831950810352|30.156881470072467| 3393.038625054409| 1289.057310595615|
|    min|                 0|Badminton Court|               0.0|               5.0|               320|                10|
|    max|                 8| Tennis Court 2|               9.9|              80.0|             10000|              3000|
+-------+------------------+----

In [0]:
%sql
SELECT name FROM country_club.Facilities
WHERE membercost > 0

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
SELECT COUNT(*) FROM country_club.Facilities
WHERE membercost IN (NULL, 0)

count(1)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT facid, name, membercost, monthlymaintenance FROM country_club.Facilities
WHERE membercost < monthlymaintenance * 0.2 

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
2,Badminton Court,0.0,50
3,Table Tennis,0.0,10
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80
7,Snooker Table,0.0,15
8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
SELECT * FROM country_club.Facilities
WHERE facid IN (1,5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
SELECT
  *,
  CASE
    WHEN monthlymaintenance > 100 THEN 'expensive'
    WHEN monthlymaintenance <= 100 THEN 'cheap'
  END as value
FROM country_club.Facilities

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,value
0,Tennis Court 1,5.0,25.0,10000,200,expensive
1,Tennis Court 2,5.0,25.0,8000,200,expensive
2,Badminton Court,0.0,15.5,4000,50,cheap
3,Table Tennis,0.0,5.0,320,10,cheap
4,Massage Room 1,9.9,80.0,4000,3000,expensive
5,Massage Room 2,9.9,80.0,4000,3000,expensive
6,Squash Court,3.5,17.5,5000,80,cheap
7,Snooker Table,0.0,5.0,450,15,cheap
8,Pool Table,0.0,5.0,400,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
SELECT * FROM country_club.Members

memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,GUEST,GUEST,GUEST,0,(000) 000-0000,null,2012-07-01 00:00:00
1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,null,2012-07-02 12:02:05
2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,null,2012-07-02 12:08:23
3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,null,2012-07-03 09:32:15
4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05
5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1,2012-07-09 10:44:09
6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,null,2012-07-15 08:52:55
7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4,2012-07-25 08:59:12
8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3,2012-07-25 16:02:35
9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6,2012-07-25 17:09:05


In [0]:
%sql
SELECT firstname, surname FROM country_club.Members
WHERE joindate = (SELECT MAX(joindate) from country_club.Members)


firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
SELECT * FROM (SELECT f.name as court_name, CONCAT(m.firstname, ' ',m.surname) as member_name, ROW_NUMBER() OVER(PARTITION BY CONCAT(m.firstname, ' ',m.surname) ORDER BY CONCAT(m.firstname, ' ',m.surname)) as RN FROM country_club.Members as m
INNER JOIN country_club.bookings as b
ON m.memid = b.memid
INNER JOIN country_club.facilities f
ON f.facid = b.facid
WHERE f.name LIKE "%Tennis%"
AND m.firstname != "GUEST") as t
WHERE t.RN = 1

court_name,member_name,RN
Table Tennis,Anna Mackenzie,1
Tennis Court 1,Anne Baker,1
Tennis Court 2,Burton Tracy,1
Tennis Court 1,Charles Owen,1
Table Tennis,Darren Smith,1
Tennis Court 1,David Farrell,1
Tennis Court 2,David Jones,1
Table Tennis,David Pinker,1
Tennis Court 1,Douglas Jones,1
Tennis Court 1,Erica Crumpet,1


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
 SELECT 
  b.bookid,
  CONCAT(m.firstname, ' ',m.surname) as member_name,
  CASE
    WHEN m.memid = 0 THEN b.slots * f.guestcost
    WHEN m.memid != 0 THEN b.slots * f.membercost
  END AS total_slots_cost
FROM country_club.Members as m
INNER JOIN country_club.bookings as b
ON m.memid = b.memid
INNER JOIN country_club.facilities f
ON f.facid = b.facid
WHERE CAST(b.starttime as DATE) in ('2012-09-14')
AND CASE
      WHEN m.memid = 0 THEN b.slots * f.guestcost
      WHEN m.memid != 0 THEN b.slots * f.membercost
    END > 30
ORDER by 3 DESC

bookid,member_name,total_slots_cost
2946,GUEST GUEST,320.0
2937,GUEST GUEST,160.0
2942,GUEST GUEST,160.0
2940,GUEST GUEST,160.0
2926,GUEST GUEST,150.0
2925,GUEST GUEST,75.0
2922,GUEST GUEST,75.0
2920,GUEST GUEST,75.0
2948,GUEST GUEST,70.0
2941,Jemima Farrell,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
SELECT * FROM (
   SELECT 
    f.name,
    CONCAT(m.firstname, ' ',m.surname) as member_name,
    CASE
      WHEN m.memid = 0 THEN b.slots * f.guestcost
      WHEN m.memid != 0 THEN b.slots * f.membercost
    END AS total_slots_cost
  FROM country_club.Members as m
  INNER JOIN country_club.bookings as b
  ON m.memid = b.memid
  INNER JOIN country_club.facilities f
  ON f.facid = b.facid
  WHERE CAST(b.starttime as DATE) in ('2012-09-14')
  ) as m
WHERE m.total_slots_cost > 30
ORDER by m.total_slots_cost DESC

name,member_name,total_slots_cost
Massage Room 2,GUEST GUEST,320.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Massage Room 1,GUEST GUEST,160.0
Tennis Court 2,GUEST GUEST,150.0
Tennis Court 2,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Tennis Court 1,GUEST GUEST,75.0
Squash Court,GUEST GUEST,70.0
Massage Room 1,Jemima Farrell,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
with cte as (
  SELECT 
    f.facid,
    CONCAT(m.firstname, ' ',m.surname) as member_name,
    CASE
      WHEN m.memid = 0 THEN b.slots * f.guestcost
      WHEN m.memid != 0 THEN b.slots * f.membercost
    END AS total_bookings_cost
  FROM country_club.Members as m
  INNER JOIN country_club.bookings as b
  ON m.memid = b.memid
  INNER JOIN country_club.facilities f
  ON f.facid = b.facid
)
select f.name, sum(c.total_bookings_cost) as revenue from cte as c
inner join country_club.facilities f
on f.facid = c.facid
group by f.name
having revenue < 1000
order by revenue DESC


name,revenue
Pool Table,270.0
Snooker Table,240.0
Table Tennis,180.0
